In [ ]:
# Imports and configuration
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2
from scipy.ndimage import label
plt.rcParams['figure.figsize'] = (7,7)
np.set_printoptions(precision=4, suppress=True)

In [ ]:
# Schrödinger–Poisson 2D toy simulation (periodic box)
N = 128              # grid size (increase to 256 for more detail)
L = 10.0             # physical box size
dx = L / N
dt = 0.01            # timestep
steps = 600          # total timesteps
snap_interval = 30   # snapshot cadence
g = 1.5              # gravity strength
hbar = 1.0; m = 1.0

# spectral grid
kx = 2*np.pi*np.fft.fftfreq(N, d=dx)
ky = 2*np.pi*np.fft.ffreq(N, d=dx)
KX, KY = np.meshgrid(kx, ky)
K2 = KX**2 + KY**2
K2[0,0] = 1.0  # avoid divide-by-zero; we set zero mode to 0 later

# physical grid
x = np.linspace(-L/2, L/2, N, endpoint=False)
y = np.linspace(-L/2, L/2, N, endpoint=False)
X, Y = np.meshgrid(x, y)

# initial condition: blobs + tiny complex noise
rng = np.random.default_rng(0)
psi = 0.001 * (rng.standard_normal((N,N)) + 1j*rng.standard_normal((N,N)))
for cx, cy, amp, s in [(-2.0,-1.0, 1.0,0.6), (2.0,1.5,0.8,0.7), (0.5,-2.2,0.9,0.5)]:
    psi += amp * np.exp(-((X-cx)**2 + (Y-cy)**2)/(2*s*s))
# normalize
norm = np.sqrt(np.sum(np.abs(psi)**2) * dx*dx)
psi /= norm

# kinetic half-step operator
expK = np.exp(-1j * (K2) * (dt/(2*m)))

def solve_poisson_from_density(rho):
    rho_k = fft2(rho)
    V_k = -g * rho_k / K2
    V_k[0,0] = 0.0  # zero mean potential (gauge)
    return np.real(ifft2(V_k))

frames = []; times = []
for t in range(steps):
    # half kinetic
    psi_k = fft2(psi); psi_k *= expK; psi = ifft2(psi_k)
    # density + potential
    rho = np.abs(psi)**2
    V = solve_poisson_from_density(rho)
    # full potential
    psi *= np.exp(-1j * V * dt)
    # half kinetic
    psi_k = fft2(psi); psi_k *= expK; psi = ifft2(psi_k)
    # occasional renormalization
    if (t % 50) == 0:
        norm = np.sqrt(np.sum(np.abs(psi)**2) * dx*dx)
        psi /= norm
    # snapshot
    if (t % snap_interval) == 0:
        frames.append(np.abs(psi)**2)
        times.append(t*dt)
        print(f'frame {len(frames)} at t={times[-1]:.2f}, max rho={frames[-1].max():.3f}')

# visualize last frame
rho = frames[-1]
plt.imshow(np.log10(rho + 1e-9), extent=[-L/2,L/2,-L/2,L/2], origin='lower', cmap='magma')
plt.colorbar(label='log10(density)')
plt.title(f'Final density t={times[-1]:.2f}')
plt.xlabel('x'); plt.ylabel('y')
plt.show()

## Diagnostic 1 — Perturbation & Recovery
Apply a local pulse to a snapshot and measure variance and max-density over a window to estimate recovery time (homeostasis).

In [ ]:
def perturb_and_measure(frames, t0_index, center, radius=3, amp=5.0, window=10):
    frames = [f.copy() for f in frames]
    N = frames[0].shape[0]
    Y, X = np.meshgrid(np.arange(N), np.arange(N))
    mask = ((X-center[0])**2 + (Y-center[1])**2) <= radius**2
    frames[t0_index][mask] *= amp
    var = [np.var(f) for f in frames[t0_index: t0_index+window]]
    peak = [np.max(f) for f in frames[t0_index: t0_index+window]]
    return np.array(var), np.array(peak)

t0 = max(0, len(frames)//2 - 1)
var, peak = perturb_and_measure(frames, t0, center=(N//2, N//2), window=min(10, len(frames)-t0))
plt.figure(); plt.plot(var, '-o'); plt.title('Variance after perturbation'); plt.xlabel('steps'); plt.ylabel('var'); plt.show()
plt.figure(); plt.plot(peak, '-o'); plt.title('Max density after perturbation'); plt.xlabel('steps'); plt.ylabel('max rho'); plt.show()

## Diagnostic 2 — Cluster Lifetimes
Detect high-density clusters in each snapshot and estimate lifetimes by simple centroid matching.

In [ ]:
def cluster_lifetimes(frames, threshold=0.15):
    labels_over_time = []
    for f in frames:
        mask = f > threshold
        lbl, _ = label(mask)
        labels_over_time.append(lbl)
    def centroid(lblarr, lab):
        ys, xs = np.where(lblarr == lab)
        if len(xs)==0: return None
        return (xs.mean(), ys.mean())
    next_id = 1; active = {}; lifetimes = []
    for t, lblarr in enumerate(labels_over_time):
        present = {};
        for lab in np.unique(lblarr):
            if lab==0: continue
            c = centroid(lblarr, lab)
            if c is None: continue
            present[lab] = c
        used = set()
        for lab, c in present.items():
            best=None; bestd=1e9; bestid=None
            for aid, (alab, acol, birth) in active.items():
                d = (c[0]-acol[0])**2 + (c[1]-acol[1])**2
                if d < bestd: bestd=d; best=aid
            if best is not None and bestd < (10**2):
                active[best] = (lab, c, active[best][2])
                used.add(best)
            else:
                active[next_id] = (lab, c, t); next_id += 1
        to_delete = [aid for aid in list(active.keys()) if aid not in used]
        for aid in to_delete:
            birth = active[aid][2]
            lifetimes.append(t - birth)
            del active[aid]
    final_t = len(frames)
    for aid, (_, _, birth) in active.items():
        lifetimes.append(final_t - birth)
    return np.array(lifetimes)

lts = cluster_lifetimes(frames, threshold=0.15)
plt.figure(); plt.hist(lts, bins=20); plt.title('Cluster lifetimes'); plt.xlabel('frames'); plt.ylabel('count'); plt.show()

## Diagnostic 3 — Time-lagged Mutual Information
Estimate information flow between two distant regions using a simple histogram-based mutual information with lag.

In [ ]:
def region_time_series(frames, region_slices):
    ts = []
    for sl in region_slices:
        arr = np.array([f[sl].mean() for f in frames])
        ts.append(arr)
    return np.array(ts)

def time_lagged_mutual_info(x, y, lag=1, bins=32):
    x = x[:-lag]; y = y[lag:]
    c_xy, _, _ = np.histogram2d(x, y, bins=bins)
    p_xy = c_xy / c_xy.sum()
    p_x = p_xy.sum(axis=1)
    p_y = p_xy.sum(axis=0)
    nz = p_xy > 0
    mi = (p_xy[nz] * np.log(p_xy[nz] / (p_x[:,None][nz] * p_y[None,:][nz]))).sum()
    return mi

# pick two regions
regions = [(slice(N//4, N//4+N//8), slice(N//4, N//4+N//8)), (slice(3*N//5, 3*N//5+N//8), slice(3*N//5, 3*N//5+N//8))]
ts = region_time_series(frames, regions)
mi_01 = time_lagged_mutual_info(ts[0], ts[1], lag=2)
print('Lagged MI between regions:', mi_01)

### Notes
- Increase `N` and `steps` for richer dynamics.
- Tune `g` to encourage collapse/mergers vs. fog.
- For more lifelike behavior, consider adding phase turbulence, Gross–Pitaevskii nonlinearity, or AMR Poisson solves (beyond this compact toy).